#Load Library 

In [3]:
## import all necessary packages
import json
import re
import pandas as pd

import nltk, string
from nltk.collocations import *

import numpy as np
import matplotlib.pyplot as plt

from pandas import *

from collections import defaultdict

import data_cleaning_hr as tc

import string
import happyfuntokenizing

SyntaxError: invalid syntax (data_cleaning_hr.py, line 250)

#Load Data

In [ ]:
with open('./data/tweets_training.json','r') as f:
    tweet_df = DataFrame(json.load(f))

#Clean Data (Handle, URL, Emoticon Conversion)

In [21]:
tc.cleanHandle(tweet_df)
tc.cleanURL(tweet_df)
tc.convertEmoticon(tweet_df)
tc.cleanRetweets(tweet_df)
tweet_df.head()

367031 handles replaced
0 urls replaced
:-*[Oo]    ->     😮 replaced    365 times
:-*\*      ->     💋 replaced    289 times
</3        ->     💔 replaced      0 times
:-*\)|\(:  ->     😃 replaced  15717 times
D:         ->     😧 replaced    208 times
<3         ->     ❤ replaced      0 times
;-*\)      ->     😉 replaced   3358 times
:-*\(|\):  ->     😞 replaced   5702 times
8\)        ->     😎 replaced    108 times
:-*>       ->     😆 replaced      0 times
:-*[PpbB]  ->     😛 replaced   1165 times
=-*\)      ->     😀 replaced    207 times
:-*\|      ->     😐 replaced     92 times
:'\(       ->     😢 replaced    339 times
:-*[/\\]   ->     😕 replaced   1960 times
;-*[PpbB]  ->     😜 replaced    330 times
>:-*\(     ->     😠 replaced      0 times
:o\)       ->     🐵 replaced      0 times
:-*D       ->     😀 replaced   1530 times
ALL replaced 31370 times
1043720 retweets processed


,id,lat,lng,text,timeStamp,user_id,retweet
0,377652254096228352,37.446100,-121.883557,hdl hey checkout the website: url,Wed Sep 11 04:38:08 +0000 2013,224874450,NaN
1,377652255346159616,34.087406,-117.462604,hdl 😪,Wed Sep 11 04:38:08 +0000 2013,312179473,NaN
2,377652262325456897,37.356131,-121.842867,i laugh a lot with that line,Wed Sep 11 04:38:10 +0000 2013,54351774,NaN
3,377652264682655744,37.364664,-122.009629,sons of anarchy is back on woop woop,Wed Sep 11 04:38:11 +0000 2013,343219606,NaN
4,377652271116722176,37.382600,-121.995000,Drinking a Fresh Squeezed IPA by hdl @ St. John's Bar &amp; Grill — url,Wed Sep 11 04:38:12 +0000 2013,1569395935,NaN


#Emoji Finder

In [22]:
try:
    # Wide UCS-4 build
    highpoints = re.compile(u'['
        u'\U0001F300-\U0001F64F'
        u'\U0001F680-\U0001F6FF'
        u'\u2600-\u26FF\u2700-\u27BF]+', 
        re.UNICODE)
except re.error:
    # Narrow UCS-2 build
    highpoints = re.compile(u'('
        u'\ud83c[\udf00-\udfff]|'
        u'\ud83d[\udc00-\ude4f\ude80-\udeff]|'
        u'[\u2600-\u26FF\u2700-\u27BF])+', 
        re.UNICODE)

In [1]:
faces = re.compile(u'['
        u'\U0001F600-\U0001F64F]',
        re.UNICODE)
# Function that takes a list of text and return text that contains just faces
def just_face(text):
    return (faces.findall(text))
# Function that take a list of text and return text with just emojis
def just_emojis(text):
    return (highpoints.findall(text))
# Function that a list of text and return the number of emojis in the text.
def count_emojis(text):
    return len(highpoints.findall(text))

# Functions to check whether there's an emoji in the text, return 1 if true, 0 if false
def is_emoji(text):
    if highpoints.search(text):
        return 1
    else:
        return 0

    # Functions to check whether there's a face emoji in the text, return 1 if true, 0 if false
def is_face(text):
    if faces.search(text):
        return 1
    else:
        return 0

NameError: name 're' is not defined

In [8]:
emoji_df["Emoji Count"] = emoji_df["text"].apply(count_emojis)
emoji_df["Text Length"] = emoji_df["text"].apply(lambda x: len(x))
emoji_df = emoji_df[['id', 'text', 'timeStamp', 'user_id', 'Emoji Count', 'Text Length']]
emoji_df.describe()

,id,user_id,Emoji Count,Text Length
count,2.082520e+05,2.082520e+05,208252.000000,208252.000000
mean,3.822541e+17,5.198651e+08,1.122659,53.161506
std,2.378889e+15,4.612050e+08,0.603226,33.193435
min,3.776523e+17,2.100000e+01,1.000000,1.000000
25%,3.801828e+17,1.499460e+08,1.000000,27.000000
50%,3.823091e+17,3.852509e+08,1.000000,46.000000
75%,3.843753e+17,7.546697e+08,1.000000,75.000000
max,3.863382e+17,1.934553e+09,70.000000,214.000000


# Exploring Emoji vs no Emoji (20% Text has Emojis, 15% has face)

###Functions to check whether it's emoji or face

In [73]:
tweet_df["is_emoji"] = tweet_df.text.apply(is_emoji)
tweet_df["is_face"] = tweet_df.text.apply(is_face)
tweet_df.head()

,id,lat,lng,text,timeStamp,user_id,is_emoji,is_face
0,377652254096228352,37.446100,-121.883557,hdl hey checkout the website: url,Wed Sep 11 04:38:08 +0000 2013,224874450,0,0
1,377652255346159616,34.087406,-117.462604,hdl 😪,Wed Sep 11 04:38:08 +0000 2013,312179473,1,1
2,377652262325456897,37.356131,-121.842867,i laugh a lot with that line,Wed Sep 11 04:38:10 +0000 2013,54351774,0,0
3,377652264682655744,37.364664,-122.009629,sons of anarchy is back on woop woop,Wed Sep 11 04:38:11 +0000 2013,343219606,0,0
4,377652271116722176,37.382600,-121.995000,Drinking a Fresh Squeezed IPA by hdl @ St. John's Bar &amp; Grill — url,Wed Sep 11 04:38:12 +0000 2013,1569395935,0,0


In [25]:
tweet_df.describe()

,id,lat,lng,user_id,is_emoji,is_face
count,1.000000e+06,1000000.000000,1000000.000000,1.000000e+06,1000000.000000,1000000.000000
mean,3.822416e+17,37.241980,-121.533430,4.961052e+09,0.208252,0.154146
std,2.370783e+15,1.333404,1.542418,4.503597e+12,0.406058,0.361089
min,3.776523e+17,12.983300,-170.296751,1.200000e+01,0.000000,0.000000
25%,3.801867e+17,37.320896,-122.350934,7.382110e+07,0.000000,0.000000
50%,3.822777e+17,37.656377,-122.043733,3.188787e+08,0.000000,0.000000
75%,3.843437e+17,37.790622,-121.785433,6.229686e+08,0.000000,0.000000
max,3.887526e+17,57.170120,77.583300,4.503597e+15,1.000000,1.000000


##Take a look at the imported tokenizer and test it

In [29]:
tok = happyfuntokenizing.TweetTokenizer(preserve_case=False)
samples = (
    u"RT @ #happyfuncoding: this is a typical Twitter tweet😖",
    u"😂😂😂 RT @Yours_Truly3x: Bitch brush yoo mouth; other Web oddities can be an &aacute;cute <em class='grumpy'>pain</em> >:(",
    u"Yay my cat is cuddling🔫 with me tonight❤ +1 (800) 123-4567, (800) 123-4567, and 123-4567 are treated as words despite their whitespace."
    )

In [31]:
for s in samples:
        print("======================================================================")
        print(s)
        tokenized = tok.tokenize(s)
        print(list(tokenized))
        print("\n")

RT @ #happyfuncoding: this is a typical Twitter tweet😖
['rt', '@', '#happyfuncoding', ':', 'this', 'is', 'a', 'typical', 'twitter', 'tweet', '😖']


😂😂😂 RT @Yours_Truly3x: Bitch brush yoo mouth; other Web oddities can be an &aacute;cute <em class='grumpy'>pain</em> >:(
['😂', '😂', '😂', 'rt', '@yours_truly3x', ':', 'bitch', 'brush', 'yoo', 'mouth', ';', 'other', 'web', 'oddities', 'can', 'be', 'an', 'ácute', "<em class='grumpy'>", 'pain', '</em>', '>:(']


Yay my cat is cuddling🔫 with me tonight❤ +1 (800) 123-4567, (800) 123-4567, and 123-4567 are treated as words despite their whitespace.
['yay', 'my', 'cat', 'is', 'cuddling', '🔫', 'with', 'me', 'tonight', '❤', '+1 (800) 123-4567', ',', '(800) 123-4567', ',', 'and', '123-4567', 'are', 'treated', 'as', 'words', 'despite', 'their', 'whitespace', '.']




#Add a new column that display just the emojis for the text

In [32]:
def emojiExtract(sent):
    return [word for word in tok.tokenize(sent) if is_emoji(word) == 1]

def textExtract(sent):
    return [word for word in tok.tokenize(sent) if is_emoji(word) == 0]

def textTokenized(sent):
    return [word for word in tok.tokenize(sent)]

def addTokenizedText(df):
    df['Tokenized'] = [textTokenized(word) for word in df.text]

def addEmojiCol(df):
    df['Emoji'] = [emojiExtract(word) for word in df.text]

def addText(df):
    df['only_Text'] = [textExtract(word) for word in df.text]

def addHashTag(df):
    df['only_HashTag'] = [re.findall(r"(#\w+)", word) for word in df.text]

In [40]:
addEmojiCol(tweet_df)
addText(tweet_df)
addTokenizedText(tweet_df)
addHashTag(tweet_df)
addTokenizedText(tweet_df)
tweet_df.head()

,id,lat,lng,text,timeStamp,user_id,retweet,Emoji,only_Text,Tokenized
0,377652254096228352,37.446100,-121.883557,hdl hey checkout the website: url,Wed Sep 11 04:38:08 +0000 2013,224874450,NaN,[],"[hdl, hey, checkout, the, website, :, url]","[hdl, hey, checkout, the, website, :, url]"
1,377652255346159616,34.087406,-117.462604,hdl 😪,Wed Sep 11 04:38:08 +0000 2013,312179473,NaN,[😪],[hdl],"[hdl, 😪]"
2,377652262325456897,37.356131,-121.842867,i laugh a lot with that line,Wed Sep 11 04:38:10 +0000 2013,54351774,NaN,[],"[i, laugh, a, lot, with, that, line]","[i, laugh, a, lot, with, that, line]"
3,377652264682655744,37.364664,-122.009629,sons of anarchy is back on woop woop,Wed Sep 11 04:38:11 +0000 2013,343219606,NaN,[],"[sons, of, anarchy, is, back, on, woop, woop]","[sons, of, anarchy, is, back, on, woop, woop]"
4,377652271116722176,37.382600,-121.995000,Drinking a Fresh Squeezed IPA by hdl @ St. John's Bar &amp; Grill — url,Wed Sep 11 04:38:12 +0000 2013,1569395935,NaN,[],"[drinking, a, fresh, squeezed, ipa, by, hdl, @, st, ., john's, bar, &, amp, ;, grill, —, url]","[drinking, a, fresh, squeezed, ipa, by, hdl, @, st, ., john's, bar, &, amp, ;, grill, —, url]"


# Word2Vec Trained Model with Twitter Corpus

In [34]:
import gensim
from gensim.models import Word2Vec

In [38]:
#emoji_model = gensim.models.Word2Vec(list(tweet_df.Tokenized))
# It might take some time to train the model. So, after it is trained, it can be saved as follows:
#emoji_model.save('emoji.embedding')
emoji_model = gensim.models.Word2Vec.load('emoji.embedding')

In [39]:
emoji_model.most_similar(positive = ['😂'], topn = 20)

[('ommmmmg', 0.7462140321731567),
 ('#dead', 0.6925972700119019),
 ('meagan', 0.6692696809768677),
 ('lmaooooooooooo', 0.6570378541946411),
 ('inseparable', 0.6502166986465454),
 ('#lrt', 0.6376644372940063),
 ('#playa', 0.6227041482925415),
 ('#dying', 0.5953817367553711),
 ('#sike', 0.5952829122543335),
 ('#coldworld', 0.5948764681816101),
 ('#blama', 0.5885862112045288),
 ('#icant', 0.5870974063873291),
 ('#workaholics', 0.5856915712356567),
 ('laffin', 0.5782320499420166),
 ('hahahahahahahahahahahahaha', 0.5772393345832825),
 ('. .. .', 0.5763115286827087),
 ('daaamn', 0.5759264826774597),
 ('omgosh', 0.5746556520462036),
 ('nikkah', 0.5687729716300964),
 ('gawwwd', 0.5650306344032288)]

In [233]:
len(tweet_df[tweet_df.text.str.contains(r'#bye')])

151

#How many Hashtags? 155979

In [454]:
tweet_df[tweet_df.only_HashTag.str.len() != 0].count(0)

id              155979
lat             155979
lng             155979
text            155979
timeStamp       155979
user_id         155979
is_emoji        155979
is_face         155979
Emoji           155979
only_Text       155979
only_HashTag    155979
Tokenized       155979
dtype: int64

#Language Cleaning

In [142]:
punctuation = string.punctuation
ex = ['“', '—', '’', ' ️', '️', '...', '”', '…', ' , , ,', '?', ' ', ' ⃣', '∞', '🆒']
for pun in [word for word in ex if word not in punctuation]:
    punctuation += pun
def isEnglish(list):
    try:
        [word.encode('ascii') for word in list if word not in punctuation]
    except Exception:
        return False
    else:
        return True

In [170]:
def cleanNonEnglish(df):
    text_list = df['only_Text'].values
    english_Boolean = [isEnglish(sent) for sent in text_list]
    return df[english_Boolean]

In [171]:
tweet_df = cleanNonEnglish(tweet_df)
len(tweet_df)